<a href="https://colab.research.google.com/github/gibsonx/jlpt_simulator/blob/dev/graphs/n3/outliner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
if 'google.colab' in str(get_ipython()):
    !git clone https://github.com/gibsonx/jlpt_simulator.git
    %cd jlpt_simulator
    !git checkout dev
    
    !apt-get install python3-dev graphviz libgraphviz-dev pkg-config
    !pip install -r requirements.txt
else:
  print('Not running on CoLab')


Not running on CoLab


In [2]:
import json
import logging
import random
import time
from spire.doc import *
from spire.doc.common import *
import pandas as pd
import yaml
from tqdm import tqdm

# from langchain_community.embeddings import XinferenceEmbeddings
from libs.LLMs import *
import datetime
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

from graphs.n3.ExamTaskHandler import ExamTaskHandler
load_dotenv()

True

In [3]:
def collect_vocabulary(file_path):
    # Read the CSV file
    data = pd.read_csv(file_path)
    # Shuffle the rows and reset the index
    words = data.iloc[:, :2].sample(frac=1).reset_index(drop=True)
    # Extract the second column (values) and convert to a single-line string
    vocab_string = ','.join(words.iloc[:, 1].astype(str).tolist())
    return vocab_string

In [4]:
# Import N3 Vocabulary
file_path = 'Vocab/n3.csv'
# Display the content of the CSV file
vocab_dict = collect_vocabulary(file_path)
with open("Vocab/topics.txt", "r", encoding="utf-8") as file:
    topics_list = [line.strip() for line in file]

In [5]:
vocab_dict

'せんこう,あたためる,まく,しゅよう,さつ,こくせき,めいし,しゅうい,ばくだい,もの,しめる,きょく,かえる,ものごと,あらた,ようてん,かん,けいえい,えん,やっかい,こうか,まなぶ,おろす,おそわる,おい,かんこう,おき,さす,やく,われわれ,いきもの,た,ゆうじょう,しよう,じゅよう,こと,かみ,きじ,いとこ,さんか,やくす,けいき,いち,かたな,はね,ははおや,こいびと,とく,おめにかかる,はい,おもわず,あたる,さく,ちがいない,まんいち,さいばん,きおん,ほしょう,ほね,しっけ,げきじょう,ちゅうしゃ,みやげ,ちゅうしん,しんこく,およぎ,けっこう,てつだい,いっしゅ,そろう,いんさつ,ちょうせん,のぞむ,もくてき,きのどく,うまる,とら,はば,きふ,ちゅうし,だんし,えんき,しんぽ,ぐん,あく,むす,けいたい,いぜん,どくとく,みとめる,すこしも,のぼる,ねんかん,おとなしい,ゆうしゅう,ようそ,しょう,ひろげる,ぶん,のぞむ,がいこう,よぼう,せい,みまい,ていでん,なかみ,ころぶ,いっしゅん,ただちに,むだ,あいする,りこう,しゅちょう,こうせい,おめでとう,わける,くむ,じゅうたい,だいきん,きれ,よあけ,せいしつ,こんにち,じっさい,つらい,よのなか,こじん,しゅうちゅう,たび,いちどに,おじぎ,げんば,たび,まずしい,きんし,ひょう,じょうしき,おこる,せいひん,つれ,ふく,ふくめる,おくれ,ちゅう,てつ,にっこう,しゅぎ,げんご,しげん,うむ,けいじ,かん,いどう,ひやす,しょうりゃく,まんぞく,あぶら,じしん,このみ,おさない,ぎいん,ひひょう,ぶっか,もどす,ぜんしん,しょり,せいめい,かぎる,おもいで,うる,しも,きれる,あな,さいてい,かえる,ぶんせき,す,ふたご,すいじゅん,まざる,かい,おう,なし,ちいき,しょくたく,しだい,ひん,せい,にる,しょうじる,ぶんや,ある,がくしゃ,せいしん,ほご,たま,もとめる,せっきょくてき,てつどう,はじまり,かかく,しどう,ゆき,こな,しょめい,ようき,まるで,こくご,とける,あせ,うわ,ほうそう,ぶじ,がくしゅう,ふうふ,こうせい,ぜん,じかに,ふうけい,ずつう,よなか,ふち,かてい,いき,いきおい,すむ,うったえる,かわ,ふ,けいい,とかい,とうあん,ひろまる,せいかく,つぐ,かこむ

#### load Models

In [6]:
from typing import List, Optional

from langchain_core.prompts import ChatPromptTemplate

from pydantic import BaseModel, Field

# instruction = """
# Section 1: Vocabulary and Grammar
# - Kanji reading (kanji_reading): 8 questions
# - Write Chinese characters (write_chinese): 6 questions
# - Word Meaning Selection (word_meaning): 11 questions
# - Synonyms substitution (synonym_substitution): 5 questions
# - word usage (word_usage): 5 questions
# - Grammar fill in the blank (sentence_grammar): 13 questions
# - Sentence sorting (sentence_sort): 5 questions
# - Grammar structure selection (sentence_structure): 4-5 questions
#
# Section 2: Reading Comprehension
# - Short passages (short_passage_read): 4 articles
# - Mid-size passages (midsize_passage_read): 2 articles
# - Long passages (long_passage_read): 1 articles
# - Information retrieval (info_retrieval): 1 articles
#
# Section 3: Listening Comprehension
# - Topic understanding (topic_understanding): 6 questions
# - Key understanding (keypoint_understanding): 6 questions
# - Summary understanding (summary_understanding): 3 questions
# - Active expression (active_expression): 4 questions
# - Immediate acknowledgment (immediate_ack): 9 questions
# """

instruction = """
Section 1: Vocabulary and Grammar
- のことばの読み方として最もよいものを、1・2・3・4から一つえらびなさい (kanji_reading): 8 questions
- このことばを漢字で書くとき、最もよいものを、１・２・３・４から一つえらびなさい (write_chinese): 6 questions

Section 2: Reading Comprehension
- Information retrieval (info_retrieval): 0 articles

Section 3: Listening Comprehension
- Topic understanding (topic_understanding): 0 questions

"""


direct_gen_outline_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            f"You are a japanese teacher. Your job is to write an outline for a JLPT Japanese-Language Proficiency Test N3 level exam paper. The words in outline must be presented in Japanese only. The complexity should be restricted to N3 level and respect japanese culture. The JLPT Japanese-Language Proficiency Test exam paper includes a mix of moderate and very difficult questions to accurately assess the test-taker's proficiency across different aspects of the language."
            f"First, evenly pick Japanese words used in different part of sentence from 'Vocabulary' for questions in Vocabulary and Grammar, ensuring 40 percent of very difficult words. At Section 2 and 3, randomly choose topics in 'TopicList' for Reading Comprehension and Listening Comprehension Sections, don't repeat to choose a same word or topic"
            f"Second, you should abide by the provided exam instruction and decide the number of questions and content in the each Section."
            f"Finally, write the outline of the examination paper in japanese and provide question topics according to the instructions."
            f"Instruction: {instruction}",
        ),
        ("user", "TopicList: {topic_list}, Vocabulary: {vocab_dict}"),
    ]
)


## Data Strcuture

In [7]:
class QuestionTopic(BaseModel):
    topic: str = Field(..., title="a vocabulary or topic hint for a question")


class Subsection(BaseModel):
    subsection_title: str = Field(..., title="subsection English word in () only from the instruction")
    description: str = Field(..., title="giving the number of questions and requirements")
    question_topics: Optional[List[QuestionTopic]] = Field(
        default_factory=list
    )

    @property
    def as_str(self) -> str:
        question_topics_str = "\n".join(
            f"- **{qt.topic}**" for qt in self.question_topics
        )
        return f"### {self.subsection_title}\n\n{self.description}\n\n{question_topics_str}".strip()

class Section(BaseModel):
    section_title: str = Field(..., title="Title of the section")
    subsections: Optional[List[Subsection]] = Field(
        default_factory=list,
        title="Titles and reason for each subsection of the JLPT exam page.",
    )

    @property
    def as_str(self) -> str:
        subsections = "\n\n".join(
            subsection.as_str for subsection in self.subsections or []
        )
        return f"## {self.section_title}\n\n{subsections}".strip()


class Outline(BaseModel):
    page_title: str = Field(..., title="Title of the JLPT exam page")
    sections: List[Section] = Field(
        default_factory=list,
        title="Titles and descriptions for each section of the JLPT exam paper.",
    )

    @property
    def as_str(self) -> str:
        sections = "\n\n".join(section.as_str for section in self.sections)
        return f"# {self.page_title}\n\n{sections}".strip()


In [8]:
# Read the topics from a file, sort them, and print the sorted list
def process_topics(file_path):
    try:
        # Read the file
        with open(file_path, 'r', encoding='utf-8') as file:
            topics = file.readlines()

        # Remove any extra whitespace or newline characters
        topics = [topic.strip() for topic in topics if topic.strip()]

        # Shuffle the topics randomly
        random.shuffle(topics)

    except FileNotFoundError:
        print("The file was not found. Please check the file path.")
    except Exception as e:
        print("An error occurred:", str(e))

    except FileNotFoundError:
        print("The file was not found. Please check the file path.")
    except Exception as e:
        print("An error occurred:", str(e))

In [9]:
# Preload all topics from the file
with open("Vocab/topics.txt", "r", encoding="utf-8") as file:
    topics_list = [line.strip() for line in file]

generate_outline_direct = direct_gen_outline_prompt | azure_llm.with_structured_output(Outline)
initial_outline = generate_outline_direct.invoke({"topic_list": topics_list, "vocab_dict": vocab_dict})

2025-05-28 14:06:27,941 - INFO - HTTP Request: POST https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"


In [10]:
from IPython.display import display, Markdown
display(Markdown(initial_outline.as_str))

# 日本語能力試験N3 模擬試験 問題用紙

## 第1部：語彙・文法

### のことばの読み方として最もよいものを、1・2・3・4から一つえらびなさい (kanji_reading)

8問。語彙リストから文中で使われる語を選び、その読み方を答える問題。難易度の高い語彙を40％含む。

- **しゅよう**
- **ばくだい**
- **じゅよう**
- **しゅちょう**
- **おもわず**
- **きょうふ**
- **ふくめる**
- **しょうりゃく**

### このことばを漢字で書くとき、最もよいものを、１・２・３・４から一つえらびなさい (write_chinese)

6問。語彙リストから選ばれた語を漢字で正しく書く。難易度の高い語彙を40％含む。

- **せんこう**
- **けいえい**
- **しゅうい**
- **しんこく**
- **ほしょう**
- **じっさい**

In [11]:
import re
outliner_json = initial_outline.model_dump_json()
data = json.loads(outliner_json)  # Replace with your actual JSON data

start_time = time.time()

output_data = {'sections': []}

for section in data['sections']:
    output_section = {'section_title': section['section_title'], 'subsections': []}

    for subsection in tqdm(section['subsections'], desc=f"Processing {section['section_title']}"):
        function_name = re.findall(r'\((.*?)\)', subsection['subsection_title'])[0]

        questions = subsection['question_topics']

        for question in tqdm(questions, desc=f"Processing {subsection['subsection_title']}"):
            handler = ExamTaskHandler()
            func = getattr(handler, function_name, None)

            if func:
                max_attempts = 2
                original_topic = question['topic']  # Optional: Track original topic
                for attempt in range(max_attempts):
                    try:
                        result = func(question['topic'])
                        question['result'] = result
                        break  # Exit on success
                    except Exception as e:
                        if attempt < max_attempts - 1:
                            # Replace topic and retry
                            question['topic'] = random.choice(topics_list)
                        else:
                            question['result'] = f"Error after {max_attempts} attempts: {str(e)}"
            else:
                question['result'] = f"Method {function_name} not found"

        output_subsection = {
            'subsection_title': subsection['subsection_title'],
            'description': subsection['description'],
            'question_topics': questions
        }
        output_section['subsections'].append(output_subsection)

    output_data['sections'].append(output_section)

# End the timer
end_time = time.time()

# Calculate the total execution time
execution_time = end_time - start_time

print(f"Total execution time: {execution_time:.2f} seconds")

Processing 第1部：語彙・文法:   0%|          | 0/2 [00:00<?, ?it/s]
Processing のことばの読み方として最もよいものを、1・2・3・4から一つえらびなさい (kanji_reading):   0%|          | 0/8 [00:00<?, ?it/s]

---WEB SEARCH---
---Generator----


2025-05-28 14:06:33,502 - INFO - HTTP Request: POST https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
2025-05-28 14:06:33,505 - INFO - Generated message: <ul>
    <li>
        体に<u>腫瘍</u>が見つかりました。
        <ul>
            <li>しゅりょう</li>
            <li>しゅよう</li>
            <li>しゅようう</li>
            <li>しゅりゅう</li>
        </ul>
    </li>
</ul>
2


---REVISOR---


2025-05-28 14:06:39,632 - INFO - HTTP Request: POST https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4.1-mini/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
2025-05-28 14:06:39,635 - INFO - Refelect message: The submitted exam question focuses on the reading of the kanji word "腫瘍," which is appropriate for the JLPT N3 level, as kanji readings and medical vocabulary can appear at this stage. The vocabulary "腫瘍" (しゅよう) is correctly chosen, and the question tests the student's ability to select the correct reading from similar-sounding options.

However, there are some points to consider for improvement:

1. Content Accuracy:
- The vocabulary choice "腫瘍" is somewhat specialized medical terminology. While it can appear in N3, it is relatively advanced. Ensure that students at this level have been exposed to this term in class or study materials.
- The incorrect options include plausible yet incorrect readings. This is good to test kanji re

...

---REVISOR---


2025-05-28 14:10:02,643 - INFO - HTTP Request: POST https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4.1-mini/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
2025-05-28 14:10:02,646 - INFO - Refelect message: Thank you for your submission. Here is a detailed critique and recommendations:

1. Content Accuracy:
- The vocabulary question focuses on the word "じっさい" and asks students to select the correct kanji among similar-looking options. This is appropriate for JLPT N3 level, as kanji recognition and usage are important.
- The answer choices include: 実在 (existence in reality), 実際 (actual, real), 実体 (substance, entity), and 実感 (real feeling). These are all valid vocabulary words, but only one fits the sentence context.
- The sentence "じっさいにやってみることが大切です。" ("Actually trying it is important.") requires the meaning "actually, in practice," which corresponds to 実際.
- The correct answer is marked as 2, which is accurate.
- The sentence is clear

--- AI Reviser feels Good Enough ---
--- Formatter ---


2025-05-28 14:10:04,212 - INFO - HTTP Request: POST https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4.1-mini/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
2025-05-28 14:10:04,215 - INFO - Formatted message: {'html_question': '<li><u>じっさい</u>にやってみることが大切です。</li>', 'correct_answer': 2, 'choices': ['実在', '実際', '実体', '実感']}

Processing このことばを漢字で書くとき、最もよいものを、１・２・３・４から一つえらびなさい (write_chinese): 100%|██████████| 6/6 [01:36<00:00, 16.05s/it]
Processing 第1部：語彙・文法: 100%|██████████| 2/2 [03:36<00:00, 108.12s/it]

Total execution time: 216.25 seconds


In [12]:
yaml_str = yaml.dump(output_data, allow_unicode=True, sort_keys=False, default_flow_style=False)
# Display as formatted YAML
display(Markdown(yaml_str))

sections:
- section_title: 第1部：語彙・文法
  subsections:
  - subsection_title: のことばの読み方として最もよいものを、1・2・3・4から一つえらびなさい (kanji_reading)
    description: 8問。語彙リストから文中で使われる語を選び、その読み方を答える問題。難易度の高い語彙を40％含む。
    question_topics:
    - topic: しゅよう
      result:
        html_question: 医者に<u>腫瘍</u>の説明を受けました。
        correct_answer: 0
        choices:
        - しゅりょう
        - しゅよう
        - しゅようう
        - しゅりゅう
    - topic: ばくだい
      result:
        html_question: 彼は<u>莫大</u>な財産を持っている。
        correct_answer: 0
        choices:
        - ぼうだい
        - ばくだい
        - まくだい
        - もうだい
    - topic: じゅよう
      result:
        html_question: この商品に<u>需要</u>があります。
        correct_answer: 1
        choices:
        - しょうよう
        - じゅよう
        - しゅうよう
        - じょうよう
    - topic: しゅちょう
      result:
        html_question: 彼は自分の意見を<u>主張</u>し続けた。
        choices:
        - しゅしょう
        - しゅちょう
        - しゅちゅう
        - しゅしょうう
        correct_answer: 2
    - topic: おもわず
      result:
        html_question: <ul><li>彼の話を聞いて<u>思わず</u>笑ってしまった。</li></ul>
        choices:
        - おもいわず
        - おもわず
        - しわず
        - おもうず
        correct_answer: 2
    - topic: きょうふ
      result:
        html_question: 暗い場所で<u>恐怖</u>を感じた。
        correct_answer: 1
        choices:
        - きょうふ
        - きょうほ
        - こうふ
        - こうほ
    - topic: ふくめる
      result:
        html_question: 次の含めるの読み方として正しいものを選んでください。この料金には、飲み物代も含める必要があります。
        choices:
        - ふくまる
        - ふくめる
        - ふくむ
        - ふくまれる
        correct_answer: 2
    - topic: しょうりゃく
      result:
        html_question: <ul><li>時間がないので説明を<u>省略</u>します。<ul><li>せいりゃく</li><li>しょうりゃく</li><li>しょうりょく</li><li>せんりゃく</li></ul></li></ul>
        choices:
        - せいりゃく
        - しょうりゃく
        - しょうりょく
        - せんりゃく
        correct_answer: 2
  - subsection_title: このことばを漢字で書くとき、最もよいものを、１・２・３・４から一つえらびなさい (write_chinese)
    description: 6問。語彙リストから選ばれた語を漢字で正しく書く。難易度の高い語彙を40％含む。
    question_topics:
    - topic: せんこう
      result:
        html_question: 大学で<u>せんこう</u>を決めました。
        correct_answer: 2
        choices:
        - 選考
        - 専攻
        - 先行
        - 閃光
    - topic: けいえい
      result:
        html_question: 彼は店の<u>けいえい</u>を始めました。
        correct_answer: 3
        choices:
        - 経栄
        - 景営
        - 経営
        - 景栄
    - topic: しゅうい
      result:
        html_question: 家の<u>しゅうい</u>を掃除しました。
        correct_answer: 3
        choices:
        - 修理
        - 集会
        - 周囲
        - 習慣
    - topic: しんこく
      result:
        html_question: 最近、環境汚染が<u>しんこく</u>になっています。
        correct_answer: 2
        choices:
        - 申告
        - 深刻
        - 新設
        - 親告
    - topic: ほしょう
      result:
        html_question: この製品には１年間の<u>ほしょう</u>が付いています。
        choices:
        - 保障
        - 補償
        - 保証
        - 保賞
        correct_answer: 3
    - topic: じっさい
      result:
        html_question: <li><u>じっさい</u>にやってみることが大切です。</li>
        correct_answer: 2
        choices:
        - 実在
        - 実際
        - 実体
        - 実感


In [13]:
output_data

{'sections': [{'section_title': '第1部：語彙・文法',
   'subsections': [{'subsection_title': 'のことばの読み方として最もよいものを、1・2・3・4から一つえらびなさい (kanji_reading)',
     'description': '8問。語彙リストから文中で使われる語を選び、その読み方を答える問題。難易度の高い語彙を40％含む。',
     'question_topics': [{'topic': 'しゅよう',
       'result': {'html_question': '医者に<u>腫瘍</u>の説明を受けました。',
        'correct_answer': 0,
        'choices': ['しゅりょう', 'しゅよう', 'しゅようう', 'しゅりゅう']}},
      {'topic': 'ばくだい',
       'result': {'html_question': '彼は<u>莫大</u>な財産を持っている。',
        'correct_answer': 0,
        'choices': ['ぼうだい', 'ばくだい', 'まくだい', 'もうだい']}},
      {'topic': 'じゅよう',
       'result': {'html_question': 'この商品に<u>需要</u>があります。',
        'correct_answer': 1,
        'choices': ['しょうよう', 'じゅよう', 'しゅうよう', 'じょうよう']}},
      {'topic': 'しゅちょう',
       'result': {'html_question': '彼は自分の意見を<u>主張</u>し続けた。',
        'choices': ['しゅしょう', 'しゅちょう', 'しゅちゅう', 'しゅしょうう'],
        'correct_answer': 2}},
      {'topic': 'おもわず',
       'result': {'html_question': '<ul><li>彼の話を聞いて<u>思わず<

In [14]:
def render_to_html(sections):
    html = '<html><body style="text-align: left;">\n'
    for section in sections:
        html += f'<h1>{section.get("section_title", "")}</h1>\n'
        for subsection in section.get("subsections", []):
            html += f'<h2>{subsection.get("subsection_title", "")}</h2>\n'
            if "description" in subsection:
                html += f'<p>{subsection["description"]}</p>\n'
            for idx, qtopic in enumerate(subsection.get("question_topics", []),1):
                # html += f'<h3>{qtopic.get("topic", "")}</h3>\n'
                result = qtopic.get("result", {})
                if isinstance(result, dict):
                    if "html_article" in result:
                        html += result["html_article"] + "\n"
                    if "html_question" in result:
                        html += f'<p><strong>{idx} . {result["html_question"]}</strong></p>\n'
                        if "choices" in result:
                            correct = result.get("correct_answer", -1)
                            html += '<ul>\n'
                            for idx, choice in enumerate(result["choices"], 1):
                                if idx == correct:
                                    html += f"<li>{idx}. <b>{choice}</b> <span style='color:green;'>(correct)</span></li>\n"
                                else:
                                    html += f"<li>{idx}. {choice}</li>\n"
                            html += '</ul>\n'
                        if "explanation" in result:
                            html += f'<p><em>Explanation: {result["explanation"]}</em></p>\n'
                    if "questions" in result and isinstance(result["questions"], list):
                        for q in result["questions"]:
                            html += f'<p><strong>Q: {q["question"]}</strong></p>\n'
                            if "choices" in q:
                                correct = q.get("correct_answer", -1)
                                html += '<ul>\n'
                                for idx, choice in enumerate(q["choices"], 1):
                                    if idx == correct:
                                        html += f"<li><b>{choice}</b> <span style='color:green;'>(correct)</span></li>\n"
                                    else:
                                        html += f"<li>{choice}</li>\n"
                                html += '</ul>\n'
                            if "explanation" in q:
                                html += f'<p><em>Explanation: {q["explanation"]}</em></p>\n'
                    if "questions" in result and isinstance(result["questions"], dict):
                        q = result["questions"]
                        html += f'<p><strong>Q: {q["question"]}</strong></p>\n'
                        if "choices" in q:
                            correct = q.get("correct_answer", -1)
                            html += '<ul>\n'
                            for idx, choice in enumerate(q["choices"], 1):
                                if idx == correct:
                                    html += f"<li><b>{choice}</b> <span style='color:green;'>(correct)</span></li>\n"
                                else:
                                    html += f"<li>{choice}</li>\n"
                            html += '</ul>\n'
                        if "explanation" in q:
                            html += f'<p><em>Explanation: {q["explanation"]}</em></p>\n'
    html += '</body></html>'
    return html

In [15]:
html_output = render_to_html(output_data['sections'])
display(Markdown(html_output))

<html><body style="text-align: left;">
<h1>第1部：語彙・文法</h1>
<h2>のことばの読み方として最もよいものを、1・2・3・4から一つえらびなさい (kanji_reading)</h2>
<p>8問。語彙リストから文中で使われる語を選び、その読み方を答える問題。難易度の高い語彙を40％含む。</p>
<p><strong>1 . 医者に<u>腫瘍</u>の説明を受けました。</strong></p>
<ul>
<li>1. しゅりょう</li>
<li>2. しゅよう</li>
<li>3. しゅようう</li>
<li>4. しゅりゅう</li>
</ul>
<p><strong>2 . 彼は<u>莫大</u>な財産を持っている。</strong></p>
<ul>
<li>1. ぼうだい</li>
<li>2. ばくだい</li>
<li>3. まくだい</li>
<li>4. もうだい</li>
</ul>
<p><strong>3 . この商品に<u>需要</u>があります。</strong></p>
<ul>
<li>1. <b>しょうよう</b> <span style='color:green;'>(correct)</span></li>
<li>2. じゅよう</li>
<li>3. しゅうよう</li>
<li>4. じょうよう</li>
</ul>
<p><strong>4 . 彼は自分の意見を<u>主張</u>し続けた。</strong></p>
<ul>
<li>1. しゅしょう</li>
<li>2. <b>しゅちょう</b> <span style='color:green;'>(correct)</span></li>
<li>3. しゅちゅう</li>
<li>4. しゅしょうう</li>
</ul>
<p><strong>5 . <ul><li>彼の話を聞いて<u>思わず</u>笑ってしまった。</li></ul></strong></p>
<ul>
<li>1. おもいわず</li>
<li>2. <b>おもわず</b> <span style='color:green;'>(correct)</span></li>
<li>3. しわず</li>
<li>4. おもうず</li>
</ul>
<p><strong>6 . 暗い場所で<u>恐怖</u>を感じた。</strong></p>
<ul>
<li>1. <b>きょうふ</b> <span style='color:green;'>(correct)</span></li>
<li>2. きょうほ</li>
<li>3. こうふ</li>
<li>4. こうほ</li>
</ul>
<p><strong>7 . 次の含めるの読み方として正しいものを選んでください。この料金には、飲み物代も含める必要があります。</strong></p>
<ul>
<li>1. ふくまる</li>
<li>2. <b>ふくめる</b> <span style='color:green;'>(correct)</span></li>
<li>3. ふくむ</li>
<li>4. ふくまれる</li>
</ul>
<p><strong>8 . <ul><li>時間がないので説明を<u>省略</u>します。<ul><li>せいりゃく</li><li>しょうりゃく</li><li>しょうりょく</li><li>せんりゃく</li></ul></li></ul></strong></p>
<ul>
<li>1. せいりゃく</li>
<li>2. <b>しょうりゃく</b> <span style='color:green;'>(correct)</span></li>
<li>3. しょうりょく</li>
<li>4. せんりゃく</li>
</ul>
<h2>このことばを漢字で書くとき、最もよいものを、１・２・３・４から一つえらびなさい (write_chinese)</h2>
<p>6問。語彙リストから選ばれた語を漢字で正しく書く。難易度の高い語彙を40％含む。</p>
<p><strong>1 . 大学で<u>せんこう</u>を決めました。</strong></p>
<ul>
<li>1. 選考</li>
<li>2. <b>専攻</b> <span style='color:green;'>(correct)</span></li>
<li>3. 先行</li>
<li>4. 閃光</li>
</ul>
<p><strong>2 . 彼は店の<u>けいえい</u>を始めました。</strong></p>
<ul>
<li>1. 経栄</li>
<li>2. 景営</li>
<li>3. <b>経営</b> <span style='color:green;'>(correct)</span></li>
<li>4. 景栄</li>
</ul>
<p><strong>3 . 家の<u>しゅうい</u>を掃除しました。</strong></p>
<ul>
<li>1. 修理</li>
<li>2. 集会</li>
<li>3. <b>周囲</b> <span style='color:green;'>(correct)</span></li>
<li>4. 習慣</li>
</ul>
<p><strong>4 . 最近、環境汚染が<u>しんこく</u>になっています。</strong></p>
<ul>
<li>1. 申告</li>
<li>2. <b>深刻</b> <span style='color:green;'>(correct)</span></li>
<li>3. 新設</li>
<li>4. 親告</li>
</ul>
<p><strong>5 . この製品には１年間の<u>ほしょう</u>が付いています。</strong></p>
<ul>
<li>1. 保障</li>
<li>2. 補償</li>
<li>3. <b>保証</b> <span style='color:green;'>(correct)</span></li>
<li>4. 保賞</li>
</ul>
<p><strong>6 . <li><u>じっさい</u>にやってみることが大切です。</li></strong></p>
<ul>
<li>1. 実在</li>
<li>2. <b>実際</b> <span style='color:green;'>(correct)</span></li>
<li>3. 実体</li>
<li>4. 実感</li>
</ul>
</body></html>

In [16]:
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
with open(f"output/JLPT_{timestamp}.html", "w", encoding="utf-8") as file:
    file.write(html_output)

In [17]:
# Create a Document object
document = Document()
# Add a section to the document
section = document.AddSection()
# Set the page margins to 72 points (72 points = 1 inch)
section.PageSetup.Margins.All = 72

# Add a paragraph to the section
paragraph = section.AddParagraph()

# Add the HTML string to the paragraph
paragraph.AppendHTML(html_output)

# Save the result document to a DOCX file
document.SaveToFile(f"output/JLPT_{timestamp}.docx", FileFormat.Docx2016)
# Or save the result document to a DOC file
# document.SaveToFile("HtmlStringToDoc.doc", FileFormat.Doc)

document.Close()